In [4]:
# Google Hash
# Team The Hoddle Warriors
# Feb 20 2020
# Updated Feb 27 2020 - 1

import itertools
import time
import math


##############
# Functions  #
##############


def total_books_given_library_id_and_days(id, days):
    books = libraries[id][0]
    first_shipping_day = libraries[id][1]
    daily_limit = libraries[id][2]
    remaining_days = days - first_shipping_day
    days_to_completion = books//daily_limit

    if remaining_days < 1:
        return 0

    if books % daily_limit != 0: # add one for ...
        days_to_completion += 1

    if days_to_completion <= remaining_days:
        return books
    else:
        return remaining_days * daily_limit

def all_pos_books_shipped_given_library_id_and_days(id, days):
    books_to_send = total_books_given_library_id_and_days(id, days)
    total_available_books = libraries[id][0]
    available_books = libraries[id][3:]

    if books_to_send == 0:
        return []
    elif books_to_send == total_available_books:
        return available_books
    else:
        temp = list(itertools.combinations(available_books, books_to_send))
        return temp

def get_score_given_books(books):
    total = 0
    for book in books:
        total += book_scores[book]
    return total

def num_days_to_ship_all(id):
    setup = libraries[id][1]
    shipping_days = math.ceil(libraries[id][0]/libraries[id][2])
    return setup + shipping_days

def highest_yielding_library_given_days_and_existing_books(book_set, days, set_of_library_ids):
    best_score = 0
    best_set = []
    best_id = -1
    new_best_flag = 0
    for id in set_of_library_ids:
        list_of_lists = all_pos_books_shipped_given_library_id_and_days(id, days)
        if not list_of_lists:
            continue
        if type(list_of_lists[0]) == int:
            list_of_lists = [list_of_lists]
        best_list = []
        for list_of_books in list_of_lists:
            pos = list(book_set | set(list_of_books))
            score = get_score_given_books(list_of_books)
            if score > best_score:
                best_id = id
                best_set = list_of_books
                best_score = score
    return best_id, best_set


def highest_yielding_library_given_days_and_existing_books_2(book_set, days, set_of_library_ids):
    best_score = 0
    best_set = []
    best_id = -1
    new_best_flag = 0
    for id in set_of_library_ids:
        set_of_books, score = get_best_books_given_used_books(id, book_set, days, timing_bias =  False)
        if score > best_score:
            best_id = id
            best_set = set_of_books
            best_score = score
    return best_id, best_set


def simple_search():
    full_library_set = set(range(0, LIBRARIES))
    remaining_libraries = full_library_set
    days_remaining = DAYS
    all_books_already_sent = set()
    while remaining_libraries and days_remaining > 0:
        print(f'days remaining: {days_remaining}')
        id, list_of_books = highest_yielding_library_given_days_and_existing_books(all_books_already_sent, days_remaining, remaining_libraries)

        if id == -1:
            # all libraries have been exhausted. nothing yields > 0
            return 0

        remaining_libraries = remaining_libraries - {id}
        all_books_already_sent = all_books_already_sent | set(list_of_books)

        days_remaining -= libraries[id][1]
        to_save_library_list.append(id)
        to_save_books.append(list(list_of_books))

def num_books_shipped_given(id, books, days):
    num_books = len(books)
    first_shipping_day = libraries[id][1]
    daily_limit = libraries[id][2]
    remaining_days = days - first_shipping_day
    days_to_completion = num_books//daily_limit

    if remaining_days < 1:
        return 0

    if num_books % daily_limit != 0: # add one for ...
        days_to_completion += 1

    if days_to_completion <= remaining_days:
        return num_books
    else:
        return remaining_days * daily_limit

def get_best_books_given_used_books(id, viable_books, days_remaining, timing_bias = True):
    if id in dic_library_to_books_ordered_by_value:
        besties, score = dic_library_to_books_ordered_by_value[id]
        besties = [i for i in besties if i in viable_books]
    else:
        viable_books = viable_books & set(libraries[id][3:])
        besties = [i for i in books_by_value if i in viable_books]

    days_to_start = libraries[id][1] # ignore daily rate
    days_to_complete = len(besties)/libraries[id][2] + libraries[id][1]
    books_shipped = num_books_shipped_given(id, besties, days_remaining)

    if timing_bias == False:
        days_to_start = 1



    # TESTING. Trying to mess with score by squaring days_to_start
    # Consider the median delay time ?
    # days remaining?
    # combine days to start and days to finish?
    # different methods depending on if library will give all books or not (early vs late)

    besties = besties[:books_shipped]
    score = sum([dic_book_id_to_value[i] for i in besties]) / days_to_start 
    dic_library_to_books_ordered_by_value[id] = (besties, score)

    return set(besties), score


# imperfect, BOTTOM UP approach to creating best library
def initialize_library_set():
    viable_books = set(range(BOOKS))
    lib_stack = []
    list_of_book_lists = []
    rem_libraries = set(range(LIBRARIES))
    days_remaining = DAYS
    progress_pointer = 0

    while days_remaining > 0:
        # initialize
        best_score = 0
        best_id = -1
        best_books = []

        if((1 - days_remaining/DAYS)*100 > progress_pointer):
            print(f'{progress_pointer}% complete. {days_remaining} days remain of {DAYS}.')
            progress_pointer += 10 # started at one before

        if not rem_libraries:
            break

        for id in rem_libraries:
            # check if possible to beat best score. Move on if impossible to beat.
            # this could be made more functional. For now, it is just used for score checking
            if id in dic_library_to_books_ordered_by_value:
                besties, score = dic_library_to_books_ordered_by_value[id]
                besties = [i for i in besties if i in viable_books]
                if score <= best_score:
                    continue

            new_books, new_score = get_best_books_given_used_books(id, viable_books, days_remaining)
            if new_score > best_score:
                best_score = new_score
                best_id = id
                best_books = new_books
        if best_score != 0:
            lib_stack.append(best_id)
            viable_books = viable_books - best_books
            days_remaining -= libraries[best_id][1]
            rem_libraries = rem_libraries - {best_id}
            list_of_book_lists.append(best_books)
        else:
            days_remaining = -1

    print(f'100% complete. 0 days remain of {DAYS}.')

    return lib_stack, set(range(BOOKS)) - viable_books, list_of_book_lists



def second_run_through(old_lib_stack, old_list_of_book_lists, old_used_books):
    new_list_of_book_lists = []
    days_remaining = DAYS
    used_books = set()
    min_days = min([library[1] for library in libraries])
    has_days_surplus = 1
    length = len(old_lib_stack)

    # determine which libraries give all their books
    # load of values in new_list_of_book_lists
    for i, id in enumerate(old_lib_stack):
        books_sent = total_books_given_library_id_and_days(id, days_remaining)
        # if the library yields all its books, show it in new list. Otherwise, keep same
        if books_sent == libraries[id][0]:
          new_list_of_book_lists.append(libraries[id][3:])
        else:
          new_list_of_book_lists.append(old_list_of_book_lists[i])
          has_days_surplus = 0
        days_remaining -= libraries[id][1]

    if has_days_surplus:
      days_remaining = DAYS
      days_surplus = 100
      for id in lib_stack:
          temp = days_remaining - num_days_to_ship_all(id)
          if temp < days_surplus:
              days_surplus = temp
          days_remaining -= libraries[id][1]
    else:
      days_surplus = 0

    days_remaining = DAYS
    viable_libraries = set(i for i in range(LIBRARIES)) - set(old_lib_stack)
    indicator = 0

    final_lib_stack = []
    final_list_of_book_lists = []
    established_set_left = set()
    established_set_right = set(old_used_books)

    for i, id in enumerate(old_lib_stack):
      # Print Progress as a percentage
      if(i/length*100 > indicator):
        print(f'{int(i/length*100)}% complete. {i} of {len(lib_stack)} libraries processed.')
        indicator+=1

      # break out of loop if adding libraries is impossible
      if days_remaining < min_days:
          break

      # make current library viable so we can include it in the search
      viable_libraries = viable_libraries | {id}
      allowable_days = libraries[id][1] + days_surplus
      restricted_libraries = set(a for a in viable_libraries if libraries[a][1] <= allowable_days)

      # get all viable books by removing the used books from our lists

      established_set_right = established_set_right - old_list_of_book_lists[i]
      viable_books = set(range(BOOKS)) - established_set_left - established_set_right

      new_id, new_books = highest_yielding_library_given_days_and_existing_books_2(viable_books, days_remaining, restricted_libraries)
      if new_id == -1:
          del new_list_of_book_lists[i] 
          viable_libraries = viable_libraries - {id}
          continue # we must continue (vs break) because some of the libraries may still contribute
      
      # update surplus size
      days_surplus += libraries[id][1] - libraries[new_id][1]

      final_list_of_book_lists.append(list(new_books))
      viable_libraries = viable_libraries - {new_id}
      final_lib_stack.append(new_id)
      days_remaining -= libraries[new_id][1]
      established_set_left = established_set_left | new_books

      # flush dictionaries
      dic_library_to_books_ordered_by_value = {}

    length = len(final_lib_stack)
    used_books = set()
    unused_books = set(range(BOOKS))
    # one final run through, cleaning up the list.
    for i in range(length):
      final_list_of_book_lists[i] = unused_books & set(final_list_of_book_lists[i])
      unused_books = unused_books - final_list_of_book_lists[i]

    used_books = set(range(BOOKS)) - unused_books

    return final_lib_stack, used_books, final_list_of_book_lists


#########
# MAIN #
#########

# load filenames
FILENAMES = []
OUTFILENAMES = []
alphabet_letters = "abcdef"
# alphabet_letters = "ab" # testing purposes
for c in alphabet_letters:
    FILENAMES.append(c + ".txt")
    OUTFILENAMES.append(c +".out")


total_score = 0
# Start main loop
for in_filename, out_filename in zip(FILENAMES, OUTFILENAMES):
    # load data
    initial_vals_list = []
    print(f'\n\n\n********************** Beginning to process {in_filename} **********************\n')
        

    # Prep Data #

    # will use these to store data to print
    to_save_library_list = []
    to_save_books= []

    # load data
    with open(in_filename, 'r') as f:
        for line in f:
            initial_vals_list.append(line)

    BOOKS, LIBRARIES, DAYS = [int(c) for c in initial_vals_list[0].split()]
    initial_vals_list = initial_vals_list[1:]
    book_scores = [int(c) for c in initial_vals_list[0].split()]
    initial_vals_list = initial_vals_list[1:]
    libraries = []

    for i in range(0,LIBRARIES):
        # amount of books, signup_time, daily shipping limit
        # books
        temp_vals_0 = [int(c) for c in initial_vals_list[0].split()]
        temp_vals_1 = [int(c) for c in initial_vals_list[1].split()]
        libraries.append(temp_vals_0 + temp_vals_1)
        initial_vals_list = initial_vals_list[2:]

    print(f'\n----- Finished loading data -----\n')

    # create a dictionary of book id to value
    dic_book_id_to_value = {i:score for i, score in enumerate(book_scores)}
    books_by_value = list(sorted(dic_book_id_to_value, key=dic_book_id_to_value.get, reverse=True))

    dic_of_libraries = {}
    dic_library_to_books_ordered_by_value = {}
    print(f'\n----- Finished preprocessing -----\n')


    # Process Data #

    # create a dictionary of {library set --> book set, score
    lib_stack, used_books, list_of_book_lists = initialize_library_set()
    print(f'\n----- Finished initializing library set -----\n')

    # clear dics
    dic_library_to_books_ordered_by_value = {}

    # Second Run Through
    lib_stack, used_books, list_of_book_lists = second_run_through(lib_stack, list_of_book_lists, used_books)
    print(f'\n----- Finished second run through of library set -----\n')


    # get score
    score = sum([book_scores[i] for i in used_books])
    total_score += score

    print(f'Results:')
    print(f'Expected score: {score} for {in_filename}')

    # Create Output
    to_save_library_list = lib_stack
    to_save_books = list_of_book_lists


    # OUTPUT  #

    # Create Output -- This could be removed if variables changed
    to_save_library_list = lib_stack
    to_save_books = list_of_book_lists

    with open(out_filename, 'w') as file:
        file.write(f'{len(to_save_library_list)}\n')
        for i in range(len(to_save_books)):
            id = to_save_library_list[i] #assume same length
            file.write(f'{id} {len(to_save_books[0])}\n')
            temp = [str(i) for i in to_save_books[0]]
            file.write(" ".join(temp) + "\n")
            to_save_books = to_save_books[1:]

print(f'\n----- Finished Everything -----\n')
print(f'Expected Total Score: {total_score}')





********************** Beginning to process a.txt **********************


----- Finished loading data -----


----- Finished preprocessing -----

0% complete. 5 days remain of 7.
10% complete. 2 days remain of 7.
100% complete. 0 days remain of 7.

----- Finished initializing library set -----

50% complete. 1 of 2 libraries processed.

----- Finished second run through of library set -----

Results:
Expected score: 21 for a.txt



********************** Beginning to process b.txt **********************


----- Finished loading data -----


----- Finished preprocessing -----

0% complete. 999 days remain of 1000.
10% complete. 898 days remain of 1000.
20% complete. 793 days remain of 1000.
30% complete. 695 days remain of 1000.
40% complete. 589 days remain of 1000.
50% complete. 499 days remain of 1000.
60% complete. 391 days remain of 1000.
70% complete. 292 days remain of 1000.
80% complete. 187 days remain of 1000.
90% complete. 97 days remain of 1000.
100% complete. 0 days rem